In [1]:
import numpy as np 
from sklearn.linear_model import LinearRegression 
import torch 
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset , TensorDataset ,DataLoader 
from torch.utils.data.dataset import random_split 
from torch.utils.tensorboard import SummaryWriter


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('fivethirtyeight')

In [3]:
%run -i data_preparation/v0.py
%run  -i model_configuration/v0.py

In [5]:
# %load model_training/v0.py

torch.manual_seed(42)
n_epochs = 100 
for epoch in range(n_epochs):
    #set the model to train
    model.train()

    #compute the model prediction
    yhat = model(x_train_tensor)

    #step-2 loss 
    loss = loss_fn(yhat , y_train_tensor)

    #compute gradients 
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()


In [6]:
#higher order function 
# means it does either 2 things , takes function as inputs or  return function
def exponentiation_builder(exponent):
    def skeleton_exponentiation(x):
        return x ** exponent

    return skeleton_exponentiation
        

In [7]:
square = exponentiation_builder(2)
cube = exponentiation_builder(3)
square(3)

9

In [8]:
cube(3)

27

In [9]:
def make_train_step(model , loss_fn , optimizer):
    def perform_train_step_fn(x ,y ):
        model.train()

        ###### model prediction
        yhat = model(x_train_tensor)

    ########## loss
        loss = loss_fn(yhat , y)

        #compute gradient
        loss.backward()

        ###### upgrade parameters
        optimizer.step()
        optimizer.zero_grad()

        return loss.item()

    return perform_train_step_fn

In [10]:
%run -i data_preparation/v0.py

In [15]:
%%writefile model_configuration/v1.py
import torch 
import torch.nn as nn
import torch.optim as optim 

#define model configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr = 0.01
#creae a model and sent to device
model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(), lr = lr)

#define mse
loss_fn = nn.MSELoss(reduction= 'mean')

def make_train_step(model , loss_fn , optimizer):
    def train_step(x , y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat , y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    return train_step

train_step_fn = make_train_step(model , loss_fn , optimizer)
        
#create a tranning step for our function
train_step_fn = make_train_step(model , loss_fn , optimizer)


Overwriting model_configuration/v1.py


In [16]:
%run -i model_configuration/v1.py

In [13]:
%%writefile model_training/v1.py
import torch 

n_epochs = 100 
losses = []

for epoch in range(n_epochs):
    loss = train_step_fn(x_train_tensor , y_train_tensor)
    losses.append(loss)



Overwriting model_training/v1.py


In [14]:
%run -i model_configuration/v1.py